## 코드

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

In [3]:
# plt.rc('font', family='NanumGothicOTF') # For MacOS
plt.rc('font', family='NanumGothic') # For Windows
mpl.rcParams['axes.unicode_minus'] = False
pd.set_option('display.max_columns', None)

In [4]:
raw_path = './csv/raw'
preprocessed_path = './csv/preprocessed'

In [5]:
df1 = pd.read_csv(f'{raw_path}/data_2017.csv',engine = 'python')
df2 = pd.read_csv(f'{raw_path}/data_2018.csv')

In [6]:
frames = [df1, df2]
df = pd.concat(frames)
df.to_csv(f'{preprocessed_path}/data_1718.csv')

In [7]:
df = df.rename(columns={"연령대코드(5세단위)":"연령대 코드", 
                        "신장(5Cm단위)":"신장", 
                        "체중(5Kg단위)":"체중",
                        "식전혈당(공복혈당)":"식전혈당",
                        "(혈청지오티)AST":"AST",
                        "(혈청지오티)ALT":"ALT",})

In [8]:
df = df.drop(['기준년도', '가입자일련번호','시도코드', '시력(좌)', '시력(우)', '청력(좌)', '청력(우)','구강검진수검여부',
       '치아우식증유무', '결손치유무', '치아마모증유무', '제3대구치(사랑니)이상', '치석', '데이터공개일자'], axis = 1)

## null 값 정리

In [9]:
df.isnull().sum()

성별코드             0
연령대 코드           0
신장               0
체중               0
허리둘레           680
수축기혈압         5749
이완기혈압         5748
식전혈당          5957
총콜레스테롤      667287
트리글리세라이드    667297
HDL콜레스테롤    667298
LDL콜레스테롤    677034
혈색소           5966
요단백          15162
혈청크레아티닌       5949
AST           5947
ALT           5948
감마지티피         5951
흡연상태           378
음주여부        355479
dtype: int64

In [10]:
df.drop(df[df['AST'] == 9999].index, inplace=True)
df.drop(df[df['감마지티피'] == 999].index, inplace=True)
df.loc[df['음주여부'].isnull(), '음주여부'] = 0

## 혈당에 따른 당뇨여부  

In [11]:
#식전혈당이 없을 시 드랍
df = df.loc[df['식전혈당'].notnull()]

#당뇨병: 공복 혈당치 126 mg/dL 이상 당뇨병의 전단계: 공복 혈당치 100 mg/dL 이상
#저혈당: 혈당치 126 mg/dL 미만
df.loc[df['식전혈당'] < 100, '식전혈당'] = 0
df.loc[(df['식전혈당'] >= 100) & (df['식전혈당'] < 126), '식전혈당'] = 1
df.loc[df['식전혈당'] >= 126, '식전혈당'] = 2

## 복부비만

In [12]:
#비정상적인 값 배제
df = df.loc[df['허리둘레']<140]
df = df.loc[df['허리둘레']>40]

#국내의 복부비만 기준은 남자는 90cm 이상, 여자의 경우 85cm 이상
df_obesity = df[['허리둘레','성별코드']].copy()

df_obesity.loc[(df_obesity['허리둘레']>= 90) & (df_obesity['성별코드']==1), '복부비만'] = 1
df_obesity.loc[(df_obesity['허리둘레']>= 85) & (df_obesity['성별코드']==2), '복부비만'] = 1
df_obesity['복부비만'] = df_obesity['복부비만'].fillna(0)

df['복부비만'] = df_obesity['복부비만']

## BMI

In [13]:
df_weight = df[['신장', '체중']].copy()

#체중(kg)을 신장(m)의 제곱으로 나눈 값(체중(kg)/신장(m2))
temp = df_weight['신장']/100
df_bmi = round(df_weight['체중'] / (temp * temp), 1)

#세계보건기구 아시아태평양지역과 대한비만학회
#BMI < 25 kg/m2 : 저체중+정상 0
#BMI ≥ 25 kg/m2 : 과체중 1
#BMI ≥ 30 kg/m2 : 비만 2
df_bmi.loc[df_bmi < 25] = 0
df_bmi.loc[(df_bmi >= 25) & (df_bmi < 30)] = 1
df_bmi.loc[df_bmi >= 30] = 2

df['비만여부'] = df_bmi

df = df.drop(['체중', '신장'], axis = 1)

## 고혈압

In [14]:
df_pressure = df[['이완기혈압', '수축기혈압']].copy()

# 정상: 0 / 120~139/80~89 전단계: 1/  140~159/90~99 1기:2 / 160/100 2기:3
df_pressure.loc[(df_pressure['수축기혈압'] < 120) & (df_pressure['이완기혈압'] < 80), '고혈압 여부'] = 0
df_pressure.loc[((df_pressure['수축기혈압'] >= 120) & (df_pressure['수축기혈압'] < 140))|
                ((df_pressure['이완기혈압'] >= 80) & (df_pressure['이완기혈압'] < 90)), '고혈압 여부'] = 1
df_pressure.loc[((df_pressure['수축기혈압'] >= 140) & (df_pressure['수축기혈압'] < 160))|
                ((df_pressure['이완기혈압'] >= 90) & (df_pressure['이완기혈압'] < 99)), '고혈압 여부'] = 2
df_pressure.loc[(df_pressure['수축기혈압'] >=160) | (df_pressure['이완기혈압'] >= 100), '고혈압 여부'] = 3

df['고혈압'] = df_pressure['고혈압 여부']
df = df.drop(['수축기혈압', '이완기혈압'], axis = 1)

## 최종

In [15]:
df.to_csv(f'{preprocessed_path}/data_preprocessed_with_na.csv', index=False)

In [16]:
df = df.dropna(axis = 0, how='any')

df_without_cholesterol = df.drop(['HDL콜레스테롤', 'LDL콜레스테롤', '트리글리세라이드', '총콜레스테롤'], axis = 1)
df.to_csv(f'{preprocessed_path}/data_preprocessed_without_cholesterol.csv', index=False)

In [17]:
weight = 0.1
invalid_cholesterol_idx = []
for index, row in df.iterrows():
    calculated = row['HDL콜레스테롤'] + row['LDL콜레스테롤'] + (row['트리글리세라이드']/5)
    total = row['총콜레스테롤']
    if (total < (1 - weight) * calculated or (1 + weight) * calculated < total):
        invalid_cholesterol_idx.append(index)
        
df = df.drop(invalid_cholesterol_idx)

In [18]:
df.to_csv(f'{preprocessed_path}/data_preprocessed.csv', index=False)

In [19]:
df.head()

,성별코드,연령대 코드,허리둘레,식전혈당,총콜레스테롤,트리글리세라이드,HDL콜레스테롤,LDL콜레스테롤,혈색소,요단백,혈청크레아티닌,AST,ALT,감마지티피,흡연상태,음주여부,복부비만,비만여부,고혈압
0,1,8,90.0,0.0,193.0,92.0,48.0,126.0,17.1,1.0,1.0,21.0,35.0,40.0,1.0,1.0,1.0,1.0,1.0
1,1,7,89.0,1.0,228.0,121.0,55.0,148.0,15.8,1.0,0.9,20.0,36.0,27.0,3.0,0.0,0.0,0.0,1.0
2,1,9,91.0,0.0,136.0,104.0,41.0,74.0,15.8,1.0,0.9,47.0,32.0,68.0,1.0,0.0,1.0,1.0,1.0
3,1,11,91.0,0.0,201.0,106.0,76.0,104.0,17.6,1.0,1.1,29.0,34.0,18.0,1.0,0.0,1.0,1.0,2.0
4,1,11,80.0,1.0,199.0,104.0,61.0,117.0,13.8,1.0,0.8,19.0,12.0,25.0,1.0,0.0,0.0,0.0,1.0
